In [11]:
import cv2
import os
import numpy as np
import pathlib

cascade_path = pathlib.Path(cv2.__file__).parent.absolute() / "data/haarcascade_frontalface_default.xml"
clf = cv2.CascadeClassifier(str(cascade_path))

dir = "./smaller-data"
for filename in os.listdir(dir):
    filepath = (os.path.join(dir, filename))
    name = filename.split(".")
    name = name[0]

    videoFile = np.load(filepath)
    colorImages = videoFile['colorImages']
    boundingBox = videoFile['boundingBox']
    landmarks2D = videoFile['landmarks2D']

    height = colorImages.shape[0]
    width = colorImages.shape[1]
    size = (width, height)
    # out = cv2.VideoWriter(os.path.join('./videos',f'{name}.mp4'), cv2.VideoWriter_fourcc(*'h264'), 25, size, isColor=False)
    for i in range(colorImages.shape[-1]):
        viss = []
        img = np.ones((height, width, 3), np.uint8)
        img1 = colorImages[:, :, :, i]
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = img1

        gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        faces = clf.detectMultiScale(
            gray,
            scaleFactor = 1.1,
            minNeighbors = 5,
            minSize = (30, 30),
            flags = cv2.CASCADE_SCALE_IMAGE
        )

        for (x, y, w, h) in faces:
            cv2.rectangle(img1, (x, y), (x + w, y + h), (255, 255, 0), 2)

        bb = boundingBox[:, :, i]
        for mark in landmarks2D[:, :, i]:
            cv2.circle(img, (int(mark[0]), int(mark[1])), 1, (255, 255, 255))

        bbox_true = [int(bb[0, 0]), int(bb[0, 1]), int(bb[3, 0]), int(bb[3, 1])]
        cv2.rectangle(img,
                        (int(bb[0, 0]), int(bb[0, 1])), 
                        (int(bb[3, 0]), int(bb[3, 1])),
                        (0, 0, 255), 1)

        combine = np.hstack((img, img1, img1, img1))
        # out.write(img1)
        cv2.imshow('video', combine)
        c = cv2.waitKey(1)
        if c == 27:
            break
    # out.release()
    cv2.destroyAllWindows()